In [18]:
from functools import partial
import math
import pickle
import pandas as pd
import geopandas as gpd
from ortools.constraint_solver import pywrapcp
from ortools.constraint_solver import routing_enums_pb2
import numpy as np
import pickle
import torch
import argparse
import math
import numpy as np
import random
import copy
import matplotlib.pyplot as plt
import pandas
import geopy.distance
from time import time
import osmnx as ox
import networkx as nx
from tqdm import tqdm
from geopy.distance import geodesic
import multiprocessing as mp
import folium
import psycopg2
import openpyxl
import pandas as pd
from tabulate import tabulate
import datetime
import pydeck as pdk
from IPython.display import display
from geopy.geocoders import Nominatim
import geocoder
import numpy as np
from matplotlib.colors import rgb2hex
import matplotlib.pyplot as plt
from geopy.geocoders import GoogleV3
from datetime import datetime, time as dt_time
from openpyxl import load_workbook
from openpyxl.styles import PatternFill
from openpyxl.worksheet.datavalidation import DataValidation
import requests
from requests.exceptions import RequestException
import time

In [19]:
# Define the path to the .pkl file
pkl_file_path = '/content/drive/MyDrive/VRP ORTools/VRP100-new.pkl'

# Load the data from the .pkl file
with open(pkl_file_path, 'rb') as handle:
    loaded_data = pickle.load(handle)

In [20]:
vehicle_capacities = 50
num_reloads = 7
num_vehicles = 12
max_distance = 10000

In [21]:
def matrix_expansion(tt_matrix, num_reloads, max_distance):
    # Calculate the new size of the matrix after adding reload nodes
    new_size = tt_matrix.shape[0] + num_reloads

    # Create a new matrix of the appropriate size, initialized to zero
    _tt_matrix = torch.zeros((new_size, new_size), dtype=tt_matrix.dtype)

    # Copy the existing tt_matrix values for non-depot nodes
    _tt_matrix[num_reloads+1:, num_reloads+1:] = tt_matrix[1:, 1:]

    # Set the travel times between reload nodes and the depot to max_distance
    _tt_matrix[:num_reloads+1,:num_reloads+1] = max_distance

    # The travel times from reload nodes to all other nodes (except other reloads and the depot)
    # should be the same as from the depot to those nodes
    for reload_node in range(1, num_reloads+1):
        _tt_matrix[reload_node, num_reloads+1:] = tt_matrix[0, 1:]
        _tt_matrix[num_reloads+1:, reload_node] = tt_matrix[1:, 0]

    # The travel times from the depot to all other nodes (except other reloads) remains unchanged
    _tt_matrix[0, num_reloads+1:] = tt_matrix[0, 1:]
    _tt_matrix[num_reloads+1:, 0] = tt_matrix[1:, 0]

    # Set the diagonal to zero (self-travel is zero time)
    torch.diagonal(_tt_matrix).fill_(0)
    # Return the expanded matrix
    return _tt_matrix

In [23]:
def create_data_model(instance, vehicle_capacities):
    """Stores the data for the problem."""
    data = {}
    data['time_matrix'] = (matrix_expansion(instance['tdtt'][:, :, 1] * 3, num_reloads, max_distance)).to(torch.int).tolist()
    demands = instance['demand'] * vehicle_capacities
    demands = demands.to(torch.int)
    demands = demands.tolist()
    # demands = [0] + [-vehicle_capacities] * num_reloads + demands[:]
    data['demands'] = [0] + [-vehicle_capacities] * num_reloads + demands
    data['vehicle_capacities'] = [vehicle_capacities] * num_vehicles
    data['num_vehicles'] = num_vehicles
    data['num_locations'] = len(data['time_matrix'])
    data['depot'] = 0
    data['vehicle_maximum_time'] = 12 * 60
    return data

In [22]:
def create_distance_evaluator(data):
    """Creates callback to return distance between points."""
    _distances = {}
    # precompute distance between location to have distance callback in O(1)
    for from_node in range(data['num_locations']):
        _distances[from_node] = {}
        for to_node in range(data['num_locations']):
            if from_node == to_node:
                _distances[from_node][to_node] = 0
            # Forbid start/end/reload node to be consecutive.
            elif from_node in range(num_reloads + 1) and to_node in range(num_reloads + 1):
                _distances[from_node][to_node] = data['vehicle_maximum_time']
            else:
                _distances[from_node][to_node] = data['time_matrix'][from_node][to_node]

    def distance_evaluator(manager, from_node, to_node):
        """Returns the manhattan distance between the two nodes"""
        return _distances[manager.IndexToNode(from_node)][manager.IndexToNode(
            to_node)]

    return distance_evaluator

In [ ]:
def add_distance_dimension(routing, manager, data, distance_evaluator_index):
    """Add Global Span constraint"""
    distance = 'Distance'
    routing.AddDimension(
        distance_evaluator_index,
        0,  # null slack
        data['vehicle_maximum_time'],  # maximum distance per vehicle
        True,  # start cumul to zero
        distance)
    distance_dimension = routing.GetDimensionOrDie(distance)
    # Try to minimize the max distance among vehicles.
    # /!\ It doesn't mean the standard deviation is minimized
    distance_dimension.SetGlobalSpanCostCoefficient(100)

In [ ]:
def create_demand_evaluator(data):
    """Creates callback to get demands at each location."""
    _demands = data['demands']

    def demand_evaluator(manager, from_node):
        """Returns the demand of the current node"""
        return _demands[manager.IndexToNode(from_node)]

    return demand_evaluator

In [ ]:
def add_capacity_constraints(routing, manager, data, demand_evaluator_index):
    """Adds capacity constraint"""
    vehicle_capacity = data['vehicle_capacities']
    capacity = 'Capacity'
    routing.AddDimension(
        demand_evaluator_index,
        vehicle_capacity[0],
        vehicle_capacity[0],
        True,  # start cumul to zero
        capacity)

    # Add Slack for reseting to zero unload depot nodes.
    # e.g. vehicle with load 10/15 arrives at node 1 (depot unload)
    # so we have CumulVar = 10(current load) + -15(unload) + 5(slack) = 0.
    capacity_dimension = routing.GetDimensionOrDie(capacity)
    # Allow to drop reloading nodes with zero cost.
    for node in range(1, num_reloads + 1):
        node_index = manager.NodeToIndex(node)
        routing.AddDisjunction([node_index], 0)

In [ ]:
def add_max_distance_dimension(routing, manager, data, distance_evaluator_index):
    """Add Global Span cost function to minimize the maximum distance traveled by any vehicle."""
    distance = 'Distance'
    maximum_distance = 10000  # You can adjust this max distance based on your specific problem constraints
    routing.AddDimension(
        distance_evaluator_index,
        0,  # no slack
        maximum_distance,  # maximum distance per vehicle
        True,  # start cumul to zero
        distance)
    distance_dimension = routing.GetDimensionOrDie(distance)
    # Aim to minimize the maximum distance traveled among all vehicles
    distance_dimension.SetGlobalSpanCostCoefficient(100)

In [ ]:
def create_time_evaluator(data):
    """Creates callback to get total times between locations."""

    def service_time(data, node):
        """Gets the service time for the specified location."""
        return 0

    def travel_time(data, from_node, to_node):
        """Gets the travel times between two locations."""
        # if from_node == to_node:
        #     travel_time = 0
        # else:
        #     travel_time = data['time_matrix'][from_node][to_node]
        return data['time_matrix'][from_node][to_node]

    _total_time = {}
    # precompute total time to have time callback in O(1)
    for from_node in range(data['num_locations']):
        _total_time[from_node] = {}
        for to_node in range(data['num_locations']):
            if from_node == to_node:
                _total_time[from_node][to_node] = 0
            else:
                _total_time[from_node][to_node] = int(
                    service_time(data, from_node) + travel_time(
                        data, from_node, to_node))

    def time_evaluator(manager, from_node, to_node):
        """Returns the total time between the two nodes"""
        return _total_time[manager.IndexToNode(from_node)][manager.IndexToNode(
            to_node)]

    return time_evaluator

In [ ]:
def print_solution(data, manager, routing, assignment):
    """Prints assignment on console"""
    print(f'Objective: {assignment.ObjectiveValue()}')
    total_distance = 0
    total_load = 0
    total_time = 0
    capacity_dimension = routing.GetDimensionOrDie('Capacity')
    time_dimension = routing.GetDimensionOrDie('Time')
    dropped = []
    for order in range(6, routing.nodes()):
        index = manager.NodeToIndex(order)
        if assignment.Value(routing.NextVar(index)) == index:
            dropped.append(order)
    print(f'dropped orders: {dropped}')
    for reload in range(1, 6):
        index = manager.NodeToIndex(reload)
        if assignment.Value(routing.NextVar(index)) == index:
            dropped.append(reload)
    print(f'dropped reload stations: {dropped}')

    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = f'Route for vehicle {vehicle_id}:\n'
        distance = 0
        while not routing.IsEnd(index):
            load_var = capacity_dimension.CumulVar(index)
            time_var = time_dimension.CumulVar(index)
            plan_output += ' {0} Load({1}) Time({2},{3}) ->'.format(
                manager.IndexToNode(index),
                assignment.Value(load_var),
                assignment.Min(time_var), assignment.Max(time_var))
            previous_index = index
            index = assignment.Value(routing.NextVar(index))
            distance += routing.GetArcCostForVehicle(previous_index, index,
                                                     vehicle_id)
        load_var = capacity_dimension.CumulVar(index)
        time_var = time_dimension.CumulVar(index)
        plan_output += ' {0} Load({1}) Time({2},{3})\n'.format(
            manager.IndexToNode(index),
            assignment.Value(load_var),
            assignment.Min(time_var), assignment.Max(time_var))
        # plan_output += f'Distance of the route: {distance}m\n'
        plan_output += f'Load of the route: {assignment.Value(load_var)}\n'
        plan_output += f'Time of the route: {assignment.Value(time_var)} min\n'
        print(plan_output)
        total_distance += distance
        total_load += assignment.Value(load_var)
        total_time += assignment.Value(time_var)
    # print('Total Distance of all routes: {}m'.format(total_distance))
    print('Total Load of all routes: {}'.format(total_load))
    print('Total Time of all routes: {} min'.format(total_time))

In [ ]:
data = create_data_model(loaded_data[1], vehicle_capacities)

manager = pywrapcp.RoutingIndexManager(data['num_locations'],
                                        data['num_vehicles'], data['depot'])

# Create Routing Model
routing = pywrapcp.RoutingModel(manager)

# Define weight of each edge
distance_evaluator_index = routing.RegisterTransitCallback(
    partial(create_distance_evaluator(data), manager))
routing.SetArcCostEvaluatorOfAllVehicles(distance_evaluator_index)

# Add Distance constraint to minimize the longuest route
add_max_distance_dimension(routing, manager, data, distance_evaluator_index)

# Add Capacity constraint
demand_evaluator_index = routing.RegisterUnaryTransitCallback(
    partial(create_demand_evaluator(data), manager))
add_capacity_constraints(routing, manager, data, demand_evaluator_index)

# # Add Time Window constraint
# time_evaluator_index = routing.RegisterTransitCallback(
#     partial(create_time_evaluator(data), manager))
# add_time_window_constraints(routing, manager, data, time_evaluator_index)

# Setting first solution heuristic (cheapest addition).
search_parameters = pywrapcp.DefaultRoutingSearchParameters()
search_parameters.first_solution_strategy = (
    routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
search_parameters.local_search_metaheuristic = (
    routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH)
search_parameters.time_limit.FromSeconds(300)

# Solve the problem.
solution = routing.SolveWithParameters(search_parameters)
if solution:
    print_solution(data, manager, routing, solution)
else:
    print("No solution found !")